In [18]:
import chromedriver_autoinstaller
from selenium import webdriver
from pysqlite import Sqlite
import time
import os

# Helpers
def remove_invalid_chars(string):
    return "".join([c for c in string if c.isalpha() ])

def create_folder(story_name, root="stories"):
    folder = os.path.join(root, story_name )
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    return folder

def save_chapter_text(folder, chap_id, chap_text):
    filename = os.path.join(folder, f"chapter_{chap_id}.txt")
    with open(filename, 'w') as f:
        f.write(chap_text)

In [19]:
db = Sqlite.init("stories_data.db")

True

In [2]:
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()

'C:\\Users\\H P\\Envs\\scrapperenv\\lib\\site-packages\\chromedriver_autoinstaller\\103\\chromedriver.exe'

In [3]:
driver = webdriver.Chrome(options=opt)

## Access Website

In [33]:
url     = "https://boxnovel.com/"
driver.get(url)

### Extract Stories

In [34]:
stories     = driver.find_elements_by_class_name("item-summary")
print       (len(stories) )

20


C:\Users\H P\Envs\scrapperenv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  """Entry point for launching an IPython kernel.


In [35]:
story_links = [(s.find_element_by_tag_name("a").get_attribute("href"), s.find_element_by_tag_name("a").text,
                s.find_element_by_class_name("chapter").text,
                s.find_element_by_class_name("chapter-item").find_element_by_class_name("post-on").text) for s in stories] 
print       ("story-links -> ", len(story_links) )

C:\Users\H P\Envs\scrapperenv\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")
C:\Users\H P\Envs\scrapperenv\lib\site-packages\selenium\webdriver\remote\webelement.py:446: UserWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  warnings.warn("find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead")


story-links ->  20


## Chapter Scrapping

In [ ]:
for story in story_links:
    link, name, lastchap, lastupdate = story
    # extract story info
    driver.get       (link)
    time.sleep       (3)
    key              = remove_invalid_chars( name )
    
    # move to first page
    read_first       = driver.find_element_by_id("btn-read-last")
    read_first.click ()
    time.sleep       (3)
    
    # create folder to store data
    folder_name = create_folder(story_name=key)
    chap_id     = 1
    while True:
        retry = 0
        while retry < 5:
            try:
                content      = driver.find_element_by_class_name("entry-content")
                chapter_text = content.find_element_by_class_name("text-left")

                try:
                    chapter_name = chapter_text.find_element_by_tag_name("h1").text
                except:
                    chapter_name = chapter_text.find_element_by_tag_name("h3").text
                print("chapter name -> ", chapter_name)

                ps           = chapter_text.find_elements_by_tag_name("p")
                ps_text      = "\n".join( [p.text for p in ps] )
                ps_text      = chapter_name + "\n" + ps_text
                print("length of content -> ", len(ps_text) )

                save_chapter_text(folder_name, chap_id, ps_text)
                chap_id      += 1
                break
            except:
                print("failed to fetch content- trying again...")
                time.sleep(5)
                retry += 1

        retry = 0
        while retry < 3:
            try:
                footer       = driver.find_element_by_id("manga-reading-nav-foot")

                next_page    = footer.find_element_by_xpath("//div[@class='nav-links']//div[@class='nav-next ']//a[@class='btn next_page']")
                next_page.click()
                time.sleep(2)
                retry = 0
                break
            except:
                retry += 1
                time.sleep(3)

        if retry == 3:
            break
    
    db.save("story", {"key":key, "name":name, "chapters":chap_id, "lastchapter":lastchap, "lastupdated":lastupdate, "storylink":link, "websource":url})
